Notebook to select genes and species
====================================

### Import yspecies utility classes and enums ###

In [1]:
from typing import *
from yspecies import *
from yspecies.enums import *
from yspecies.dataset import *
from yspecies.misc import *

## Parameters cell ##
Parameters are overiddent by papermill when run inside DVC stages

In [2]:
# Parameters
drop_species = ["Bos_grunniens", "Ornithorhynchus_anatinus", "Erinaceus_europaeus"]
min_samples_per_species = 2
tissues = ["Heart", "Liver", "Kidney", "Brain", "Lung", "Skin", "Blood"]
drop_samples = ["SRR032135","SRR5928355","SRR5928357","SRR5928359","SRR306394","SRR306397","SRR306400" ,"SRR8309416","SRR8309417","SRR9066909","SRR5767265","SRR5767266","SRR5767268","SRR5767269","SRR5767270","SRR867587" ,"SRR908037" ,"SRR908038" ]

orthology =  Orthology.one2oneplus_directed.name
reference_species = "Homo_sapiens"
animal_class = AnimalClass.mammals.name
na_threshold = 0.1

reading parameters

In [3]:
ortho = Orthology[orthology]
cl = AnimalClass[animal_class]

#### Load dependencies ####

In [4]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#### Load pathes ####

In [6]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

## Load gene tables ##

In [7]:
load_path = locations.input.genes.by_class  / cl.name.capitalize() / (ortho.name + ".tsv")
load_path

PosixPath('../data/input/genes/by_animal_class/Mammalia/one2oneplus_directed.tsv')

### Load species and samples ###

In [8]:
species = load_table(locations.input.species)
species_in_class = species[species.animal_class.eq(cl.name)]
print("Selected "+ cl.name + " class with " + str(species_in_class.shape[0]) + " species" )
species_in_class.head(3)

Selected Mammalia class with 48 species


,species,common_name,animal_class,order,family,lifespan,ensembl_url,mass_g,metabolic_rate,temperature,temperature_kelvin,gestation_days,taxon,female_maturity_days,male_maturity_days,litters_per_year,inter_birth_interval,birth_weight_g,weaning_weight_g
10,Homo_sapiens,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan
11,Loxodonta_africana,African bush elephant,Mammalia,Proboscidea,Elephantidae,65.000,https://www.ensembl.org/Loxodonta_africana,nan,nan,36.200,309.350,670.000,taxon#9785,1.000,3650.000,0.200,1707.000,105000.000,nan
12,Gorilla_gorilla,Gorilla,Mammalia,Primates,Hominidae,60.100,https://www.ensembl.org/Gorilla_gorilla,nan,nan,35.500,308.650,256.000,taxon#9595,1.000,4015.000,0.300,1397.000,2061.400,nan


In [9]:
samples = load_table(locations.input.samples)
samples_ext = samples.merge(species, on="species", how="inner")
samples_in_class = samples_ext[samples_ext["animal_class"] ==  cl.name]
selected_samples = samples_in_class[~samples_in_class.run.isin(drop_samples) & ~samples_in_class.run.isin(drop_samples)]
samples_in_class.head(3)

,bioproject,series,run,species,tissue,sample_name,characteristics,sequencer,age,sex,tumor,source,study,study_title,salmon_version,library_layout,library_selection,library_strategy,lib_type,bootstrap,protocol,common_name,animal_class,order,family,lifespan,ensembl_url,mass_g,metabolic_rate,temperature,temperature_kelvin,gestation_days,taxon,female_maturity_days,male_maturity_days,litters_per_year,inter_birth_interval,birth_weight_g,weaning_weight_g
0,PRJNA247712,PRJNA247712,SRR1287653,Ailuropoda_melanoleuca,Blood,SL01,no;Model organism or animal;19;female;blood;SL...,Illumina_HiSeq_2000,19,female,no,blood,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,The giant panda blood Transcriptome,1.1.0,PAIRED,PCR,RNA-Seq,A,96,NaN,Giant panda,Mammalia,Carnivora,Ursidae,36.800,https://www.ensembl.org/Ailuropoda_melanoleuca,nan,nan,nan,nan,48.000,taxon#9646,1.500,2192.000,0.700,548.000,110.000,nan
1,PRJNA247712,PRJNA247712,SRR1287654,Ailuropoda_melanoleuca,Blood,XB01,no;Model organism or animal;12;male;blood;XB01...,Illumina_HiSeq_2000,12,male,no,blood,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,The giant panda blood Transcriptome,1.1.0,PAIRED,PCR,RNA-Seq,A,96,NaN,Giant panda,Mammalia,Carnivora,Ursidae,36.800,https://www.ensembl.org/Ailuropoda_melanoleuca,nan,nan,nan,nan,48.000,taxon#9646,1.500,2192.000,0.700,548.000,110.000,nan
2,PRJNA247712,PRJNA247712,SRR1287655,Ailuropoda_melanoleuca,Blood,XB02,no;Model organism or animal;6;female;blood;XB0...,Illumina_HiSeq_2000,6,female,no,blood,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,The giant panda blood Transcriptome,1.1.0,PAIRED,PCR,RNA-Seq,A,128,NaN,Giant panda,Mammalia,Carnivora,Ursidae,36.800,https://www.ensembl.org/Ailuropoda_melanoleuca,nan,nan,nan,nan,48.000,taxon#9646,1.500,2192.000,0.700,548.000,110.000,nan


In [10]:
by_tissue_count_initial = samples_in_class.groupby(["tissue","species"]).run.count().sort_values(ascending=False) #initial (before dropping species and tissues)
by_tissue_count_initial

tissue       species                   
Liver        Rattus_norvegicus             21
Brain        Mus_musculus                  16
Liver        Mus_musculus                  15
Brain        Rattus_norvegicus             15
Liver        Mesocricetus_auratus          14
Brain        Cavia_porcellus               14
             Homo_sapiens                  12
Liver        Monodelphis_domestica         11
             Microcebus_murinus            10
Brain        Cavia_aperea                  10
Kidney       Rattus_norvegicus              9
Brain        Ornithorhynchus_anatinus       9
Liver        Mus_caroli                     8
Brain        Pan_paniscus                   7
Heart        Monodelphis_domestica          7
Kidney       Monodelphis_domestica          7
Heart        Ornithorhynchus_anatinus       7
Brain        Monodelphis_domestica          7
Kidney       Ornithorhynchus_anatinus       7
Blood        Tursiops_truncatus             6
Lung         Monodelphis_domestica      

### Check issues with species annotations ###
Some species may need annotations (for instance lifespan is not known)

In [11]:
samples_anti = samples.merge(species, on="species", how='outer', indicator=True)
samples_species = samples_anti[['_merge','run', 'species', 'tissue', 'sample_name',
       'characteristics', 'sequencer', 'age', 'sex',"species"]].sort_values(by="_merge")
need_annotation = samples_species[samples_species["_merge"]=="left_only"]
print(need_annotation.shape)
need_annotation

(2, 10)


,_merge,run,species,tissue,sample_name,characteristics,sequencer,age,sex,species
413,left_only,SRR1552212,Notamacropus_eugenii,Liver,meu_daughter,no;Missing;1656;Model organism or animal;12 we...,Illumina_HiSeq_2000,12 weeks,female,Notamacropus_eugenii
412,left_only,SRR1041778,Notamacropus_eugenii,Liver,GSM1278059,no;individual 1;liver;liver;male,Illumina_HiSeq_2000,NaN,NaN,Notamacropus_eugenii


### Select samples ###

In [12]:
def filter_samples(samples: pd.DataFrame, tissues: List[str], drop_species: List[str], drop_samples: List[str]) -> Tuple[pd.DataFrame,pd.DataFrame]:
    if (tissues == ["all"] or tissues == [] or tissues is None):
        filter = ~samples.run.isin(drop_samples)  & ~samples.species.isin(drop_species)
    else:
        filter = ~samples.run.isin(drop_samples) & ~samples.species.isin(drop_species) & samples.tissue.isin(tissues)
    return (samples[filter].set_index("run"),samples[~filter].set_index("run") )

In [13]:
#tissues = ["Heart", "Liver", "Kidney", "Brain", "Lung"]
(samples_filtered, samples_dropped) = filter_samples(samples_in_class, tissues, drop_species, drop_samples)
if (not min_samples_per_species is None) and min_samples_per_species > 1:    
    by_species_count = samples_in_class.groupby(["species"]).run.count().sort_values(ascending=False)   
    to_drop = by_species_count[by_species_count <= min_samples_per_species].index.to_list()
    selected_samples = samples_filtered[~samples_filtered.species.isin(to_drop)]    
    extra_dropped = samples_filtered[samples_filtered.species.isin(to_drop)]
    samples_dropped = pd.concat([samples_dropped, extra_dropped])
else:
    selected_samples = samples_filtered
tab([["initial","selected", "dropped"],[samples_in_class.shape,samples_filtered.shape,samples_dropped.shape]])
selected_samples.head(3)

initial,selected,dropped
"(560, 39)","(466, 38)","(101, 38)"


,bioproject,series,species,tissue,sample_name,characteristics,sequencer,age,sex,tumor,source,study,study_title,salmon_version,library_layout,library_selection,library_strategy,lib_type,bootstrap,protocol,common_name,animal_class,order,family,lifespan,ensembl_url,mass_g,metabolic_rate,temperature,temperature_kelvin,gestation_days,taxon,female_maturity_days,male_maturity_days,litters_per_year,inter_birth_interval,birth_weight_g,weaning_weight_g
run,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SRR1287653,PRJNA247712,PRJNA247712,Ailuropoda_melanoleuca,Blood,SL01,no;Model organism or animal;19;female;blood;SL...,Illumina_HiSeq_2000,19,female,no,blood,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,The giant panda blood Transcriptome,1.1.0,PAIRED,PCR,RNA-Seq,A,96,NaN,Giant panda,Mammalia,Carnivora,Ursidae,36.800,https://www.ensembl.org/Ailuropoda_melanoleuca,nan,nan,nan,nan,48.000,taxon#9646,1.500,2192.000,0.700,548.000,110.000,nan
SRR1287654,PRJNA247712,PRJNA247712,Ailuropoda_melanoleuca,Blood,XB01,no;Model organism or animal;12;male;blood;XB01...,Illumina_HiSeq_2000,12,male,no,blood,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,The giant panda blood Transcriptome,1.1.0,PAIRED,PCR,RNA-Seq,A,96,NaN,Giant panda,Mammalia,Carnivora,Ursidae,36.800,https://www.ensembl.org/Ailuropoda_melanoleuca,nan,nan,nan,nan,48.000,taxon#9646,1.500,2192.000,0.700,548.000,110.000,nan
SRR1287655,PRJNA247712,PRJNA247712,Ailuropoda_melanoleuca,Blood,XB02,no;Model organism or animal;6;female;blood;XB0...,Illumina_HiSeq_2000,6,female,no,blood,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,The giant panda blood Transcriptome,1.1.0,PAIRED,PCR,RNA-Seq,A,128,NaN,Giant panda,Mammalia,Carnivora,Ursidae,36.800,https://www.ensembl.org/Ailuropoda_melanoleuca,nan,nan,nan,nan,48.000,taxon#9646,1.500,2192.000,0.700,548.000,110.000,nan


In [14]:
species_in_samples = selected_samples.species.drop_duplicates()#.to_list()
species_in_samples.shape
selected_species = species.set_index("species").loc[species_in_samples]
dropped_species = species_in_class[~species_in_class.species.isin(species_in_samples)].set_index("species")
print("droped species"+str(dropped_species.index.to_list()))

tab([["species_in_samples", "selected_species"],[str(species_in_samples.shape), str(selected_species.shape)]])


droped species['Pongo_pygmaeus', 'Lynx_canadensis', 'Bos_grunniens', 'Ornithorhynchus_anatinus', 'Dasypus_novemcinctus', 'Chinchilla_lanigera', 'Sarcophilus_harrisii', 'Erinaceus_europaeus']


species_in_samples,selected_species
"(40,)","(40, 18)"


In [15]:
species_in_class[~species_in_class.species.isin(species_in_samples)]

,species,common_name,animal_class,order,family,lifespan,ensembl_url,mass_g,metabolic_rate,temperature,temperature_kelvin,gestation_days,taxon,female_maturity_days,male_maturity_days,litters_per_year,inter_birth_interval,birth_weight_g,weaning_weight_g
14,Pongo_pygmaeus,Orangutan,Mammalia,Primates,Hominidae,59.000,https://www.ensembl.org/Pongo_abelii,nan,nan,nan,nan,249.000,taxon#9602,1.000,2555.000,0.200,1414.000,1736.500,nan
28,Lynx_canadensis,Canada lynx,Mammalia,Carnivora,Felidae,26.800,https://www.ensembl.org/Lynx_canadensis,nan,nan,nan,nan,63.000,taxon#61383,3.500,573.000,1.000,365.000,204.000,nan
29,Bos_grunniens,Yak,Mammalia,Artiodactyla,Bovidae,26.300,https://www.ensembl.org/Bos_grunniens,nan,nan,38.000,311.150,274.000,taxon#30521,1.000,738.000,0.800,nan,18000.000,nan
34,Ornithorhynchus_anatinus,Duck-billed platypus,Mammalia,Monotremata,Ornithorhynchidae,22.600,https://www.ensembl.org/Ornithorhynchus_anatinus,1030.300,1.931,34.000,307.150,17.000,taxon#9258,2.000,548.000,nan,365.000,nan,nan
35,Dasypus_novemcinctus,Nine-banded armadillo,Mammalia,Cingulata,Dasypodidae,22.300,https://www.ensembl.org/Dasypus_novemcinctus,3413.700,4.655,34.500,307.650,133.000,taxon#9361,4.000,365.000,1.000,nan,66.000,nan
43,Chinchilla_lanigera,Long-tailed chinchilla,Mammalia,Rodentia,Chinchillidae,17.200,https://www.ensembl.org/Chinchilla_lanigera,436.700,1.310,34.700,307.850,111.000,taxon#34839,2.000,240.000,2.000,nan,35.000,nan
44,Sarcophilus_harrisii,Tasmanian devil,Mammalia,Dasyuromorphia,Dasyuridae,13.000,https://www.ensembl.org/Sarcophilus_harrisii,6126.800,8.664,35.800,308.950,26.000,taxon#9305,3.000,730.000,1.000,365.000,0.024,nan
46,Erinaceus_europaeus,Western European hedgehog,Mammalia,Erinaceomorpha,Erinaceidae,11.700,https://www.ensembl.org/Erinaceus_europaeus,1213.500,2.434,34.000,307.150,30.000,taxon#9365,1.500,253.000,1.600,nan,14.700,nan


Count how many samples do we have by tissue and species

In [16]:
def samples_in(*cols):
    spc = by_species_count.set_index("species")
    return sum([spc.loc[c].run for c in cols])


In [17]:
by_species_tissue_count = selected_samples.sort_values(["species", "tissue"], ascending=False).groupby(["tissue", "common_name", "species"]).size()
by_species_tissue_count 

tissue  common_name                     species                   
Blood   African bush elephant           Loxodonta_africana             4
        Black snub-nosed monkey         Rhinopithecus_bieti            1
        Bottlenosed dolphin             Tursiops_truncatus             6
        Domestic cattle                 Bos_taurus                     1
        Giant panda                     Ailuropoda_melanoleuca         3
        Horse                           Equus_caballus                 1
        Rhesus monkey                   Macaca_mulatta                 4
        Wild boar                       Sus_scrofa                     6
Brain   American black bear             Ursus_americanus               2
        Black snub-nosed monkey         Rhinopithecus_bieti            1
        Brazilian guinea pig            Cavia_aperea                  10
        Chimpanzee                      Pan_troglodytes                2
        Domestic cat                    Felis_catus      

# Genes #

In [18]:
genes = load_table(locations.input.genes.by_class  / cl.name.capitalize() / (ortho.name + ".tsv"), reference_species, "str")
print(genes.shape)
genes.head()

(21924, 47)


,Loxodonta_africana,Gorilla_gorilla,Pan_troglodytes,Pongo_pygmaeus,Equus_caballus,Pan_paniscus,Tursiops_truncatus,Macaca_mulatta,Macaca_fascicularis,Macaca_nemestrina,Ailuropoda_melanoleuca,Ursus_americanus,Heterocephalus_glaber,Rhinolophus_ferrumequinum,Vombatus_ursinus,Felis_catus,Sus_scrofa,Lynx_canadensis,Bos_grunniens,Canis_lupus_familiaris,Rhinopithecus_bieti,Ovis_aries,Callithrix_jacchus,Ornithorhynchus_anatinus,Dasypus_novemcinctus,Phascolarctos_cinereus,Capra_hircus,Suricata_suricatta,Bos_taurus,Aotus_nancymaae,Otolemur_garnettii,Microcebus_murinus,Chinchilla_lanigera,Sarcophilus_harrisii,Cavia_porcellus,Erinaceus_europaeus,Tupaia_belangeri,Oryctolagus_cuniculus,Ictidomys_tridecemlineatus,Meriones_unguiculatus,Cavia_aperea,Monodelphis_domestica,Mus_musculus,Mesocricetus_auratus,Rattus_norvegicus,Mus_spicilegus,Mus_caroli
Homo_sapiens,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000242265,NaN,ENSGGOG00000040792,ENSPTRG00000023648,NaN,ENSECAG00000001603,ENSPPAG00000030053,ENSTTRG00000003114,ENSMMUG00000032524,ENSMFAG00000045560,ENSMNEG00000037924,ENSAMEG00000019304,NaN,NaN,ENSRFEG00010011916,ENSVURG00010024800,ENSFCAG00000041002,ENSSSCG00000036049,ENSLCNG00005003562,ENSBGRG00000010269,ENSCAFG00030017055;ENSCAFG00040013027;ENSCAFG0...,ENSRBIG00000013492,ENSOARG00000002475,ENSCJAG00000033086,NaN,ENSDNOG00000036759,ENSPCIG00000028966,NaN,ENSSSUG00005000112,ENSBTAG00000053735,ENSANAG00000021637,ENSOGAG00000031169,ENSMICG00000001469,ENSCLAG00000017695,NaN,NaN,NaN,NaN,ENSOCUG00000027535,ENSSTOG00000022801,ENSMUGG00000022866;ENSMUGG00000022860,ENSCAPG00000010493,NaN,ENSMUSG00000092035,ENSMAUG00000018656,NaN,ENSMSIG00000000323,MGP_CAROLIEiJ_G0028078
ENSG00000139990,ENSLAFG00000023301,ENSGGOG00000009358,ENSPTRG00000006478,ENSPPYG00000005936,ENSECAG00000010524,ENSPPAG00000042013,ENSTTRG00000013941,ENSMMUG00000019346,ENSMFAG00000003413,ENSMNEG00000028207,ENSAMEG00000014443,ENSUAMG00000010770,ENSHGLG00100003336;ENSHGLG00100017958;ENSHGLG0...,ENSRFEG00010004811,ENSVURG00010023351,ENSFCAG00000029516,ENSSSCG00000039701,ENSLCNG00005004208,ENSBGRG00000007044,ENSCAFG00030015021;ENSCAFG00040013486;ENSCAFG0...,ENSRBIG00000044703,ENSOARG00000021180,ENSCJAG00000042954,ENSOANG00000012620,ENSDNOG00000018872,ENSPCIG00000010372,ENSCHIG00000021848,ENSSSUG00005006529,ENSBTAG00000048519,ENSANAG00000030404,ENSOGAG00000008452,ENSMICG00000046667,ENSCLAG00000006604,ENSSHAG00000000665,ENSCPOG00000032392,ENSEEUG00000011616,NaN,ENSOCUG00000010281,ENSSTOG00000026551,ENSMUGG00000023917,ENSCAPG00000014053,ENSMODG00000010219,ENSMUSG00000049106,ENSMAUG00000004350,ENSRNOG00000004556,ENSMSIG00000015740,MGP_CAROLIEiJ_G0017895
ENSG00000073921,ENSLAFG00000000754,ENSGGOG00000009116,ENSPTRG00000004141,ENSPPYG00000003744,ENSECAG00000019264,ENSPPAG00000036162,ENSTTRG00000001554,ENSMMUG00000019618,ENSMFAG00000002978,ENSMNEG00000029285,ENSAMEG00000008778,ENSUAMG00000018059,ENSHGLG00100018440,ENSRFEG00010012989,ENSVURG00010006226,ENSFCAG00000029123,ENSSSCG00000014913,ENSLCNG00005007055,ENSBGRG00000024679,ENSCAFG00030009174;ENSCAFG00000004486;ENSCAFG0...,ENSRBIG00000028102,ENSOARG00000004654,ENSCJAG00000014262,ENSOANG00000013205,ENSDNOG00000038474,ENSPCIG00000018862,ENSCHIG00000015634,ENSSSUG00005014072,ENSBTAG00000001657,ENSANAG00000019901,ENSOGAG00000013950,ENSMICG00000017261,ENSCLAG00000014073,ENSSHAG00000004629,ENSCPOG00000020758,ENSEEUG00000013932,ENSTBEG00000000600,ENSOCUG00000023809,ENSSTOG00000008689,ENSMUGG00000002616,ENSCAPG00000000355,ENSMODG00000004297,ENSMUSG00000039361,ENSMAUG00000011448,ENSRNOG00000018322,ENSMSIG00000002381,MGP_CAROLIEiJ_G0029981
ENSG00000139687,ENSLAFG00000013579,ENSGGOG00000016440,ENSPTRG00000005864,ENSPPYG00000005357,ENSECAG00000014575,ENSPPAG00000002165,ENSTTRG00000004232,ENSMMUG00000007891,ENSMFAG00000004544,ENSMNEG00000030265,ENSAMEG00000003640,ENSUAMG00000011956,ENSHGLG00100008595,ENSRFEG00010000766,ENSVURG00010023466,ENSFCAG00000024529,ENSSSCG00000009401,ENSLCNG00005000165,ENSBGRG00000020100,ENSCAFG00040011728;ENSCAFG00000004

In [19]:
def gc(df: pd.DataFrame, species: pd.DataFrame = None, index_field: str = None) -> pd.DataFrame:
    frame = df.notna().sum().sort_values(ascending=False).to_frame("genes")
    result = frame.reset_index().rename( columns= {"index": "species"})
    if species is None:
        return result
    else:
        field = "common_name" if index_field is None else index_field        
        return result.merge(species, on="species", how="inner").set_index(field)

### Missing value analysis ###

In [20]:
def show_missing(df: pd.DataFrame, title: str = "missing plot", ax = None):
     return sns.heatmap(df.notnull(), cbar=False,ax=ax).set_title(title)      
  
def compare_missing(one: pd.DataFrame, two: pd.DataFrame, one_title: str, two_title: str):
    print("Missing values comparison:")
    print("before: "+str(one.shape) +" || after: "+str(two.shape))    
    print("samples lost:"+str(samples_in(drop_species)))    
    print("Figures:")
    fig, axs = plt.subplots(1,2,figsize=(20,12))
    plt.subplots_adjust(left=0.125, bottom=0.3, right=0.9, top=1.1, wspace=0.3, hspace=0.8)        
    sns.heatmap(one.notnull(), cbar=False,ax=axs[0]).set_title(one_title)    
    sns.heatmap(two.notnull(), cbar=False,ax=axs[1]).set_title(two_title)


In [21]:
#todo: fix issues with charts

### Genes with deleted species and too many NA ###

In [22]:
filtered_genes = genes.drop(columns = dropped_species.index).dropna(thresh=0.001)
filtered_genes.shape

(21882, 39)

Computing the number of not_na_values

In [23]:
not_na_number = (1-na_threshold) * filtered_genes.shape[1]
not_na_number

35.1

In [25]:
selected_genes = genes.drop(columns = dropped_species.index).dropna(thresh=not_na_number)
print("before = "+str(genes.dropna(thresh=1).shape))
print("after = "+str(selected_genes.shape))
selected_genes.head()

before = (21924, 47)
after = (12243, 39)


,Loxodonta_africana,Gorilla_gorilla,Pan_troglodytes,Equus_caballus,Pan_paniscus,Tursiops_truncatus,Macaca_mulatta,Macaca_fascicularis,Macaca_nemestrina,Ailuropoda_melanoleuca,Ursus_americanus,Heterocephalus_glaber,Rhinolophus_ferrumequinum,Vombatus_ursinus,Felis_catus,Sus_scrofa,Canis_lupus_familiaris,Rhinopithecus_bieti,Ovis_aries,Callithrix_jacchus,Phascolarctos_cinereus,Capra_hircus,Suricata_suricatta,Bos_taurus,Aotus_nancymaae,Otolemur_garnettii,Microcebus_murinus,Cavia_porcellus,Tupaia_belangeri,Oryctolagus_cuniculus,Ictidomys_tridecemlineatus,Meriones_unguiculatus,Cavia_aperea,Monodelphis_domestica,Mus_musculus,Mesocricetus_auratus,Rattus_norvegicus,Mus_spicilegus,Mus_caroli
Homo_sapiens,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000139990,ENSLAFG00000023301,ENSGGOG00000009358,ENSPTRG00000006478,ENSECAG00000010524,ENSPPAG00000042013,ENSTTRG00000013941,ENSMMUG00000019346,ENSMFAG00000003413,ENSMNEG00000028207,ENSAMEG00000014443,ENSUAMG00000010770,ENSHGLG00100003336;ENSHGLG00100017958;ENSHGLG0...,ENSRFEG00010004811,ENSVURG00010023351,ENSFCAG00000029516,ENSSSCG00000039701,ENSCAFG00030015021;ENSCAFG00040013486;ENSCAFG0...,ENSRBIG00000044703,ENSOARG00000021180,ENSCJAG00000042954,ENSPCIG00000010372,ENSCHIG00000021848,ENSSSUG00005006529,ENSBTAG00000048519,ENSANAG00000030404,ENSOGAG00000008452,ENSMICG00000046667,ENSCPOG00000032392,NaN,ENSOCUG00000010281,ENSSTOG00000026551,ENSMUGG00000023917,ENSCAPG00000014053,ENSMODG00000010219,ENSMUSG00000049106,ENSMAUG00000004350,ENSRNOG00000004556,ENSMSIG00000015740,MGP_CAROLIEiJ_G0017895
ENSG00000073921,ENSLAFG00000000754,ENSGGOG00000009116,ENSPTRG00000004141,ENSECAG00000019264,ENSPPAG00000036162,ENSTTRG00000001554,ENSMMUG00000019618,ENSMFAG00000002978,ENSMNEG00000029285,ENSAMEG00000008778,ENSUAMG00000018059,ENSHGLG00100018440,ENSRFEG00010012989,ENSVURG00010006226,ENSFCAG00000029123,ENSSSCG00000014913,ENSCAFG00030009174;ENSCAFG00000004486;ENSCAFG0...,ENSRBIG00000028102,ENSOARG00000004654,ENSCJAG00000014262,ENSPCIG00000018862,ENSCHIG00000015634,ENSSSUG00005014072,ENSBTAG00000001657,ENSANAG00000019901,ENSOGAG00000013950,ENSMICG00000017261,ENSCPOG00000020758,ENSTBEG00000000600,ENSOCUG00000023809,ENSSTOG00000008689,ENSMUGG00000002616,ENSCAPG00000000355,ENSMODG00000004297,ENSMUSG00000039361,ENSMAUG00000011448,ENSRNOG00000018322,ENSMSIG00000002381,MGP_CAROLIEiJ_G0029981
ENSG00000139687,ENSLAFG00000013579,ENSGGOG00000016440,ENSPTRG00000005864,ENSECAG00000014575,ENSPPAG00000002165,ENSTTRG00000004232,ENSMMUG00000007891,ENSMFAG00000004544,ENSMNEG00000030265,ENSAMEG00000003640,ENSUAMG00000011956,ENSHGLG00100008595,ENSRFEG00010000766,ENSVURG00010023466,ENSFCAG00000024529,ENSSSCG00000009401,ENSCAFG00040011728;ENSCAFG00000004436;ENSCAFG0...,ENSRBIG00000033901,ENSOARG00000008246,ENSCJAG00000009076,ENSPCIG00000029307,ENSCHIG00000004374,ENSSSUG00005003247,ENSBTAG00000006640,ENSANAG00000022056,ENSOGAG00000007461,ENSMICG00000001747,ENSCPOG00000008683,ENSTBEG00000014454,ENSOCUG00000016404,ENSSTOG00000000927,ENSMUGG00000019124,ENSCAPG00000014795,ENSMODG00000011168,ENSMUSG00000022105,ENSMAUG00000020865,ENSRNOG00000016029,ENSMSIG00000007265,MGP_CAROLIEiJ_G0019499
ENSG00000119977,ENSLAFG00000001639,ENSGGOG00000034939,ENSPTRG00000002786,ENSECAG00000016594,ENSPPAG00000043730,ENSTTRG00000003261,ENSMMUG00000003103,ENSMFAG00000001575,ENSMNEG00000037026,ENSAMEG00000015948,ENSUAMG00000021674,ENSHGLG00100006578,ENSRFEG00010015913,ENSVURG00010001079,ENSFCAG00000023675,ENSSSCG00000010496,ENSCAFG00000008406;ENSCAFG00040023697;ENSCAFG0...,ENSRBIG00000034323,ENSOARG00000006497,ENSCJAG00000016206,ENSPCIG00000020525,ENSCHIG00000021683,ENSSSUG00005007325,ENSBTAG00000011841,ENSANAG00000030005,ENSOGAG00000003757,ENSMICG00000015970,ENSCPOG00000021974,ENSTBEG00000010577,ENSOCUG00000005964,ENSSTOG00000007961,ENSMUGG00000008987,NaN,ENSMODG00000004976,ENSMUSG00000025008,ENSMAUG00000021808,ENSRNOG00000047112;ENSRNOG00000060994,ENSMSIG00000005758,MGP_CAROLIEiJ_G0022941
ENSG00000242866,ENSLAFG00000031348,ENSGGOG00000004658,ENSPTRG000000069

# Gene Expressions #

In [26]:
expressions = load_table(locations.input.expressions.by_class / (cl.name + ".tsv"), "reference_gene")
expressions.head(3)

,SRR1521445,SRR306838,SRR306839,SRR306840,SRR306841,SRR306842,SRR306843,SRR306844,SRR306845,SRR306847,SRR3715877,SRR5008362,SRR5120939,SRR5120940,SRR5885325,SRR5961830,SRR5961875,SRR5961972,SRR787277,SRR8702484,SRR8991100,SRR6307195,SRR6307196,SRR6307197,SRR6307204,SRR306800,SRR306801,SRR306802,SRR306803,SRR306804,SRR306805,SRR306806,SRR306807,SRR306808,SRR306809,SRR306810,SRR649365,DRR031591,SRR032135,SRR1510173,SRR1758916,SRR1758921,SRR1758922,SRR2040586,SRR306791,SRR306793,SRR095666,SRR3403827,SRR3403828,SRR4039470,SRR4039471,SRR4039473,SRR636850,SRR636900,SRR636945,SRR306826,SRR306827,SRR306828,SRR306829,SRR306831,SRR306832,SRR306833,SRR306834,SRR306835,SRR306836,SRR8750397,SRR8750398,SRR8750399,SRR3195085,SRR3195086,SRR3195096,SRR3195103,SRR3195112,SRR3195113,SRR5080320,SRR5080324,SRR5080392,SRR5080564,SRR1047652,SRR223518,SRR223519,SRR223520,SRR223521,SRR223522,SRR299126,SRR3197665,SRR5990543,SRR6007051,SRR6073386,SRR6073415,SRR1758941,SRR1758942,SRR1758943,SRR1758954,SRR1758957,SRR223512,SRR223513,SRR223514,SRR223515,SRR223516,SRR223517,SRR1759005,SRR1759006,SRR1759007,SRR1759011,SRR1759012,SRR1287653,SRR1287654,SRR1287655,SRR2308103,SRR636887,SRR636888,SRR636932,SRR636933,SRR636977,SRR636978,ERR1331676,SRR2124226,SRR306394,SRR306395,SRR306396,SRR306397,SRR306398,SRR306399,SRR306400,SRR306401,SRR306402,SRR306403,SRR306404,SRR306406,SRR1048140,SRR1048142,SRR2754983,SRR2757329,ERR1331678,ERR1331679,ERR2716205,ERR2716206,ERR2716207,ERR2716208,ERR2716211,ERR2716213,ERR2716214,SRR1334837,SRR636854,SRR636855,SRR636856,SRR636904,SRR636905,SRR636906,SRR636948,SRR636949,SRR636950,SRR3160008,SRR3160018,SRR3160041,SRR3160052,SRR3160061,SRR3194634,SRR3194682,SRR3194698,SRR3194730,SRR3194765,SRR3194791,SRR653846,SRR9720682,SRR1747117,SRR1747118,SRR1747119,SRR1747120,SRR1747121,SRR361433,SRR361434,SRR361435,SRR361436,SRR5118366,SRR5190423,SRR5190425,SRR5190427,SRR5190428,SRR5190430,SRR5190431,SRR5190489,SRR5190491,SRR5190493,SRR5190507,SRR5190508,SRR5190509,SRR636842,SRR636843,SRR636892,SRR636893,SRR636937,SRR636938,SRR1300759,SRR1300763,SRR1300765,SRR1300766,SRR1300767,SRR1300768,ERR2074874,ERR2075072,ERR2075082,ERR2075792,ERR2075986,ERR489282,ERR489283,SRR1013904,SRR1758977,SRR1758978,SRR1758979,SRR1758981,SRR1758982,SRR1758983,SRR1758984,SRR1758985,SRR3109717,SRR3109718,SRR5190444,SRR5190446,SRR5928355,SRR5928357,SRR5928359,SRR765910,SRR866213,SRR306724,SRR306726,SRR306728,SRR306729,SRR306730,SRR306731,SRR306732,SRR306733,SRR306735,SRR306737,SRR5412224,SRR5412225,SRR5412226,SRR5412227,SRR5412228,SRR5412229,SRR5412230,SRR5412231,SRR5412232,SRR5412233,SRR5412234,SRR5412235,SRR5412236,SRR553592,SRR553593,SRR553594,SRR553595,SRR649381,SRR6206908,SRR6206918,SRR1205138,SRR1205218,SRR1205222,SRR1205223,SRR1205998,SRR8708135,SRR1822406,SRR3109726,SRR3109728,SRR489494,SRR5190441,SRR5190453,SRR9024741,SRR9024746,SRR9024747,SRR9024749,SRR9024753,SRR9024755,ERR1331716,SRR1981979,SRR1981981,SRR1981987,SRR1981988,SRR3109705,SRR3109709,SRR3109710,SRR3419167,SRR4444968,SRR4444969,SRR4444970,SRR4444971,SRR636839,SRR636840,SRR636934,SRR636935,SRR924544,ERR1331694,SRR1758992,SRR1758995,SRR1758996,SRR1758997,SRR1758998,SRR4444954,SRR4444955,SRR4444956,SRR4444957,SRR4444958,SRR4444959,SRR4444960,SRR4444961,SRR4249993,ERR1331704,ERR1331711,ERR1331683,ERR1331712,ERR1331715,ERR162228,ERR162229,ERR162240,ERR162246,ERR162251,ERR162267,ERR266373,ERR266376,ERR266380,ERR266381,ERR266394,SRR1200908,SRR1200909,SRR1200911,SRR636846,SRR636847,SRR636848,SRR636896,SRR636897,SRR636898,SRR636941,SRR636942,SRR636943,SRR636852,SRR636853,SRR636902,SRR636903,SRR636946,SRR636947,ERR1331667,ERR1331707,ERR1331708,ERR1331709,SRR6261040,SRR6261041,SRR6261042,SRR6261043,ERR1331671,ERR266370,SRR1786019,SRR1789057,SRR1789059,SRR1789326,SRR1789331,SRR6293952,SRR6293953,SRR6293961,SRR6293972,SRR6293973,SRR6293975,SRR636865,SRR636866,SRR636867,SRR636913,SRR636914,SRR636915,SRR636958,SRR636959,SRR636960,SRR8309416,SRR8309417,SRR9066909,SRR9066910,SRR9066919,SRR9066929,SRR9066930,SRR

## Filtering expressions ##

In [27]:
expressions_filtered = expressions.loc[selected_genes.index].drop(columns=samples_dropped.index)
print(expressions_filtered.shape)
assert expressions_filtered.shape[0]==selected_genes.shape[0], "for all selected genes we should have expression values!"
assert expressions_filtered.shape[1]==selected_samples.shape[0], "for all selected samples we should have expression values!"
expressions_filtered.head(3)

(12243, 459)


,SRR1521445,SRR306838,SRR306839,SRR306840,SRR306841,SRR306842,SRR306843,SRR306844,SRR306845,SRR306847,SRR3715877,SRR5008362,SRR5120939,SRR5120940,SRR5885325,SRR5961830,SRR5961875,SRR5961972,SRR787277,SRR8702484,SRR8991100,SRR6307195,SRR6307196,SRR6307197,SRR6307204,SRR306800,SRR306801,SRR306802,SRR306803,SRR306804,SRR306805,SRR306806,SRR306807,SRR306808,SRR306809,SRR649365,DRR031591,SRR1510173,SRR1758916,SRR1758921,SRR1758922,SRR2040586,SRR095666,SRR3403827,SRR3403828,SRR636850,SRR636900,SRR636945,SRR306826,SRR306827,SRR306828,SRR306829,SRR306831,SRR306832,SRR306833,SRR306834,SRR306835,SRR306836,SRR8750397,SRR8750398,SRR8750399,SRR3195085,SRR3195086,SRR3195096,SRR3195103,SRR3195112,SRR3195113,SRR5080320,SRR5080324,SRR5080392,SRR5080564,SRR1047652,SRR223518,SRR223519,SRR223520,SRR223521,SRR299126,SRR5990543,SRR6007051,SRR6073386,SRR6073415,SRR1758941,SRR1758942,SRR1758943,SRR1758954,SRR1758957,SRR223512,SRR223514,SRR223515,SRR1759005,SRR1759006,SRR1759007,SRR1759011,SRR1759012,SRR1287653,SRR1287654,SRR1287655,SRR2308103,SRR636887,SRR636888,SRR636932,SRR636933,SRR636977,SRR636978,ERR1331676,SRR2124226,SRR306395,SRR306396,SRR306398,SRR306399,SRR306401,SRR306402,SRR306404,SRR306406,SRR1048140,SRR1048142,SRR2754983,SRR2757329,ERR1331678,ERR1331679,ERR2716205,ERR2716206,ERR2716207,ERR2716208,ERR2716213,ERR2716214,SRR1334837,SRR636854,SRR636855,SRR636856,SRR636904,SRR636905,SRR636906,SRR636948,SRR636949,SRR636950,SRR3160008,SRR3160018,SRR3160041,SRR3160052,SRR3160061,SRR3194634,SRR3194682,SRR3194698,SRR3194730,SRR3194765,SRR3194791,SRR653846,SRR636842,SRR636843,SRR636892,SRR636893,SRR636937,SRR636938,SRR1300759,SRR1300763,SRR1300765,SRR1300766,SRR1300767,SRR1300768,ERR2074874,ERR2075072,ERR2075082,ERR2075792,ERR2075986,ERR489282,ERR489283,SRR1013904,SRR1758977,SRR1758978,SRR1758979,SRR1758981,SRR1758982,SRR1758983,SRR1758984,SRR1758985,SRR3109717,SRR3109718,SRR5190444,SRR5190446,SRR765910,SRR1205138,SRR1205218,SRR1205222,SRR1205223,SRR1205998,SRR8708135,SRR1822406,SRR3109726,SRR3109728,SRR489494,SRR5190441,SRR5190453,SRR9024741,SRR9024746,SRR9024747,SRR9024753,SRR9024755,ERR1331716,SRR1981979,SRR1981981,SRR1981987,SRR1981988,SRR3109705,SRR3109709,SRR3109710,SRR4444968,SRR4444969,SRR4444970,SRR4444971,SRR636839,SRR636840,SRR636934,SRR636935,SRR924544,ERR1331694,SRR1758992,SRR1758995,SRR1758996,SRR1758997,SRR1758998,SRR4444954,SRR4444955,SRR4444956,SRR4444957,SRR4444958,SRR4444959,SRR4444960,SRR4444961,ERR1331683,ERR1331712,ERR1331715,ERR162228,ERR162229,ERR162240,ERR162246,ERR162251,ERR162267,ERR266373,ERR266376,ERR266380,ERR266381,ERR266394,SRR1200908,SRR1200909,SRR636846,SRR636847,SRR636848,SRR636896,SRR636897,SRR636898,SRR636941,SRR636942,SRR636943,ERR1331667,ERR1331707,ERR1331708,ERR1331709,SRR6261040,SRR6261041,SRR6261042,ERR1331671,ERR266370,SRR1786019,SRR1789057,SRR1789059,SRR1789326,SRR1789331,SRR6293952,SRR6293953,SRR6293961,SRR6293972,SRR6293973,SRR6293975,SRR636865,SRR636866,SRR636867,SRR636913,SRR636914,SRR636915,SRR636958,SRR636959,SRR636960,SRR9066910,SRR9066919,SRR9066929,SRR9066931,SRR9066934,ERR162213,ERR162217,ERR162222,ERR162245,ERR162261,ERR266353,ERR266372,ERR266378,ERR266393,ERR266398,ERR1331668,ERR1331710,ERR1331725,SRR306742,SRR306744,SRR306747,SRR306749,SRR306751,SRR306752,SRR306753,SRR306754,SRR449433,SRR449434,SRR449435,SRR449436,SRR5412205,SRR5412206,SRR5412207,SRR5412209,SRR5412210,SRR5412211,SRR5412212,SRR5412213,SRR5412215,SRR5412216,SRR5412218,SRR5952126,SRR5952127,SRR5952128,SRR5952129,SRR5952130,SRR6206899,SRR6206904,SRR6206909,SRR6206914,ERR2004718,ERR2004719,ERR2004720,SRR1653996,SRR1654032,SRR2925196,SRR2925197,SRR2925198,SRR2925199,SRR2925200,SRR2925201,SRR2925202,SRR2925203,SRR2925204,SRR2925205,SRR2925206,SRR2925207,SRR2925208,SRR2925209,SRR2925210,SRR2925212,SRR2925244,SRR2925245,SRR2925247,SRR2925248,SRR2925249,SRR2925251,SRR2925252,SRR2925253,SRR2925254,SRR2925255,SRR2925258,SRR2925259,SRR2925260,SRR2925267,SRR5115667,SRR5115668,SRR5115669,SRR5115678,SRR1549160,SRR1549162,SRR3468363,SRR346836

In [28]:
exp = expressions_filtered.T
exp.index.names = ["run"]
print(exp.shape)
exp.columns

(459, 12243)


Index(['ENSG00000139990', 'ENSG00000073921', 'ENSG00000139687',
       'ENSG00000119977', 'ENSG00000242866', 'ENSG00000135506',
       'ENSG00000162426', 'ENSG00000165995', 'ENSG00000073756',
       'ENSG00000138050',
       ...
       'ENSG00000204889', 'ENSG00000275023', 'ENSG00000275489',
       'ENSG00000278259', 'ENSG00000199158', 'ENSG00000207008',
       'ENSG00000207605', 'ENSG00000207691', 'ENSG00000207726',
       'ENSG00000208892'],
      dtype='object', name='Homo_sapiens', length=12243)

In [29]:
selected_expressions = exp
selected_expressions[selected_expressions.columns[0:10]].tail(5)

Homo_sapiens,ENSG00000139990,ENSG00000073921,ENSG00000139687,ENSG00000119977,ENSG00000242866,ENSG00000135506,ENSG00000162426,ENSG00000165995,ENSG00000073756,ENSG00000138050
run,,,,,,,,,,
ERR3350030,5.975,28.073,1.280,0.858,0.000,87.180,0.000,0.000,0.000,2.087
ERR476402,7.414,31.914,2.525,0.632,0.000,80.971,0.000,0.399,0.000,2.039
ERR476404,7.350,36.706,4.138,0.715,0.000,68.186,0.000,0.000,0.000,3.011
ERR476406,5.803,27.978,2.591,1.354,0.000,88.410,0.000,0.000,0.000,3.003
ERR476408,5.279,30.673,1.880,0.409,0.000,100.924,0.000,0.106,0.000,1.854


In [30]:
assert selected_genes.index.size == exp.shape[1], "expressions should have all genes we selected!"
assert selected_samples.shape[0] == exp.shape[0], "all expressions should have samples metadata"

# Writing selected species, samples, genes, expressions #

In [31]:
selected_species.to_csv(locations.interim.species, sep = "\t", index = True)
selected_genes.to_csv(locations.interim.genes, sep = "\t", index = True)
selected_samples.to_csv(locations.interim.samples, sep="\t", index = True)
selected_expressions.to_csv(locations.interim.expressions, sep="\t", index = True)

